In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/playing with btc

/content/drive/My Drive/Colab Notebooks/playing with btc


In [ ]:
!pip install backtrader
!pip install pyfolio
# For plotting capabilities
!pip install backtrader[plotting]

     |████████████████████████████████| 419kB 2.8MB/s 
     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
  Created wheel for pyfolio: filename=pyfolio-0.9.2-cp36-none-any.whl size=88666 sha256=5eaf73122e41e98139b50c766dc950d1c4832dae45b4058095d6fb78d1456fc2
  Stored in directory: /root/.cache/pip/wheels/0d/a8/7d/ac33fe6bf6f3db370291c9de303460d00b8f97cc40a1d80f72
  Created wheel for empyrical: filename=empyrical-0.5.5-cp36-none-any.whl size=39763 sha256=5d52606fd5790c1abdc302a3470799df863ee18430a7426f2005c7ef37baf9e1
  Stored in directory: /root/.cache/pip/wheels/ea/b2/c8/6769d8444d2f2e608fae2641833110668d0ffd1abeb2e9f3fc
Successfully built pyfolio empyrical


In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 144628 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 2.7MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437805 sha256=5a76e3ada92cefb9afcb40be9be80047030e15af0629767863b877bd97805cbf
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [ ]:
%matplotlib inline
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
import seaborn as sns
import pyfolio as pf
import talib
import datetime

from backtrader.feeds import PandasData


In [ ]:
df = pd.read_csv('bitcoin/btc_4h_2017_2020.csv')
names = ['open', 'high', 'low', 'close', 'volume', 'open_time']
df = df[names]
df

,open,high,low,close,volume,open_time
0,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 04:00:00.000
1,4333.32,4485.39,4333.32,4427.30,63.619882,2017-08-17 08:00:00.000
2,4436.06,4485.39,4333.42,4352.34,174.562001,2017-08-17 12:00:00.000
3,4352.33,4354.84,4200.74,4325.23,225.109716,2017-08-17 16:00:00.000
4,4307.56,4369.69,4258.56,4285.08,249.769913,2017-08-17 20:00:00.000
...,...,...,...,...,...,...
6775,10449.94,10502.79,10360.00,10372.90,7823.399186,2020-09-22 04:00:00.000
6776,10372.91,10503.00,10372.90,10452.26,8386.425935,2020-09-22 08:00:00.000
6777,10452.11,10520.00,10416.00,10451.07,9044.594223,2020-09-22 12:00:00.000
6778,10451.06,10506.22,10447.53,10491.97,6701.006892,2020-09-22 16:00:00.000


# ticker and the start and end dates for testing

In [ ]:
format = '%Y-%m-%d %H:%M:%S.%f'
df['open_time'] = pd.to_datetime(df['open_time'], format=format)
df.set_index('open_time', inplace=True)
df.head(2)

,open,high,low,close,volume
open_time,,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882


# Create indicators

- Exponential moving average 200 periods
- MACD
  - macd is the difference between the fast EMA and slow EMA.
  - macdsignal is the EMA of the MACD value with period signalperiod
  - macdhist computes the difference between macd and macdsignal



In [ ]:
# EMA 200
df['ema_200'] = talib.EMA(df.close, timeperiod=200)
macd, macdsignal, macdhist = talib.MACD(df.close, fastperiod=12, slowperiod=26, signalperiod=9)
df['macd_12'] = macd
df['macd_26'] = macdsignal
df

,open,high,low,close,volume,red_green,ema_200,macd_12,macd_26
open_time,,,,,,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865,1,NaN,NaN,NaN
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882,1,NaN,NaN,NaN
2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001,0,NaN,NaN,NaN
2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716,0,NaN,NaN,NaN
2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-09-22 04:00:00,10449.94,10502.79,10360.00,10372.90,7823.399186,0,10847.969380,-115.629242,-46.161465
2020-09-22 08:00:00,10372.91,10503.00,10372.90,10452.26,8386.425935,1,10844.031973,-121.093347,-61.147842
2020-09-22 12:00:00,10452.11,10520.00,10416.00,10451.07,9044.594223,0,10840.121903,-124.089290,-73.736131


In [ ]:
# class to define the columns we will provide
class SignalData(PandasData):
    """
    Define pandas DataFrame structure
    """
    OHLCV = ['open', 'high', 'low', 'close', 'volume']
    cols = OHLCV + ['ema_200', 'macd_12', 'macd_26']

    # create lines
    lines = tuple(cols)

    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params = tuple(params.items())

# Strategy

We will buy only when the price is above the 200EMA and when the macd_12 is above macd_26.

We will sell when the macd_26 is above macd_12 or if the price is below the 200EMA.

In [ ]:
# define backtesting strategy class
class Strategy(bt.Strategy):
  params = dict()
  
  def __init__(self):

    self.btc = self.datas[0]
    self.close = self.btc.close
    self.ema_200 = self.btc.ema_200
    self.macd_12 = self.btc.macd_12
    self.macd_26 = self.btc.macd_26
    self.last_id = 0

  # logging function
  def log(self, txt):
    '''Logging function'''
    dt = self.datas[0].datetime.date(0).isoformat()
    print(f'{dt}, {txt}')

  def next(self):
    # not in the market
    if not self.position:

      if self.close > self.ema_200:
        if self.macd_12 > self.macd_26:
          self.buy(exectype=bt.Order.Market)

          self.log(f'Buy at {self.close[0]}')
    
    else:
      if self.close < self.ema_200:
        self.sell(exectype=bt.Order.Market)

      if self.macd_12 < self.macd_26:
        self.sell(exectype=bt.Order.Market)


In [ ]:
# instantiate SignalData class
data = SignalData(dataname=df['2020-06'])

In [ ]:
# instantiate Cerebro, add strategy, data, initial cash, commission and pyfolio for performance analysis
cerebro = bt.Cerebro(stdstats = False)
cerebro.addstrategy(Strategy)
cerebro.adddata(data, name=ticker)
cerebro.broker.setcash(20000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

In [ ]:
# run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
backtest_result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 20000.00
2020-06-01, Buy at 10200.77
2020-06-02, Order 0 Completed Size 1 Price: 10202.71 Cost: 10202.71 Comm 10.20271
2020-06-02, Order 0 Completed Size -1 Price: 9507.69 Cost: 10202.71 Comm 9.50769
2020-06-04, Buy at 9817.64
2020-06-04, Order 0 Completed Size 1 Price: 9817.65 Cost: 9817.65 Comm 9.81765
2020-06-05, Order 0 Completed Size -1 Price: 9711.76 Cost: 9817.65 Comm 9.71176
2020-06-07, Buy at 9746.99
2020-06-08, Order 0 Completed Size 1 Price: 9746.99 Cost: 9746.99 Comm 9.74699
2020-06-09, Order 0 Completed Size -1 Price: 9681.81 Cost: 9746.99 Comm 9.68181
2020-06-09, Buy at 9721.24
2020-06-09, Order 0 Completed Size 1 Price: 9721.23 Cost: 9721.23 Comm 9.72123
2020-06-11, Order 0 Completed Size -1 Price: 9521.54 Cost: 9721.23 Comm 9.521540000000002
2020-06-13, Buy at 9473.34
2020-06-14, Order 0 Completed Size 1 Price: 9473.34 Cost: 9473.34 Comm 9.47334
2020-06-15, Order 0 Completed Size -1 Price: 9164.09 Cost: 9473.34 Comm 9.16409
2020-06-15, Order 0 